In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from numpy import genfromtxt
import pywt
from datetime import time, datetime, timedelta
import random
import itertools
import io
import json
import cvxopt

from classification import *
from kernel import *

In [3]:
Xtr = genfromtxt('../data/Xtr.csv', delimiter=',')
Ytr = genfromtxt('../data/Ytr.csv', delimiter=',')
Xte = genfromtxt('../data/Xte.csv', delimiter=',')

Xtr = np.delete(Xtr, 3072, axis=1)
Xte = np.delete(Xte, 3072, axis=1)
Ytr = Ytr[1:,1]
N = len(Ytr)

## Test new SVM

In [4]:
#features_functions = [fourier_modulus_1D_kernel, fourier_modulus_2D_kernel, fourier_phase_2D_kernel, wavelet_transform]
#lambdas = [10, 1, 0.1, 0.01]
#features_functions = [fourier_modulus_2D_kernel]
features_functions = [scattering_with_haar_wavelets]
lambdas = [0.1]
submission = True

results_cross_val = {}

for feature_function in features_functions:
    Xtr_t = feature_function(Xtr)
    for _lambda in lambdas:
        results_cross_val[(feature_function, _lambda)] = []
        for i in range(1):
            
            if submission:
                Xte_t = feature_function(Xte)
                features = Xtr_t.T
                labels = Ytr
                X = Xte_t.T
            else:
                mask_test = random.sample(range(0, 5000), 1000)
                mask_train =  [i for i in range(5000) if i not in mask_test]

                Xtr_train = Xtr_t[mask_train, :].T
                Xtr_test = Xtr_t[mask_test, :].T
                Ytr_train = Ytr[mask_train]
                Ytr_test = Ytr[mask_test]
                
                features = Xtr_train
                labels = Ytr_train
                X = Xtr_test

            t1 = datetime.now()
            alphas, bias = one_versus_all_SVM(features, labels, _lambda=_lambda)
            print 'model fitted'
            prediction = predict_SVM(alphas, bias, features, X)
            t2 = datetime.now()
            print t2-t1
            
            if submission == False:
                well_classified = 0
                for i in range(len(prediction)):
                    if prediction[i] == Ytr_test[i]:
                         well_classified+=1
                results_cross_val[(feature_function, _lambda)].append(well_classified)
                print 'lambda = ', _lambda, ', good classification rate = ', float(well_classified)/len(Ytr_test)
        
                

     pcost       dcost       gap    pres   dres
 0: -5.3889e+02 -4.7071e+01  4e+04  2e+02  2e-14
 1: -4.6283e+01 -1.4958e+01  2e+03  1e+01  2e-14
 2: -4.7802e+00 -1.0804e+01  1e+02  7e-01  4e-15
 3: -1.2011e+00 -8.6734e+00  1e+01  3e-02  6e-15
 4: -9.9201e-01 -2.3301e+00  1e+00  2e-03  1e-14
 5: -9.8654e-01 -1.1366e+00  2e-01  2e-04  2e-15
 6: -9.8765e-01 -1.1310e+00  1e-01  2e-04  2e-15
 7: -9.9176e-01 -1.0755e+00  8e-02  6e-05  1e-15
 8: -9.9589e-01 -1.0257e+00  3e-02  1e-05  1e-15
 9: -9.9669e-01 -1.0205e+00  2e-02  9e-06  1e-15
10: -9.9833e-01 -1.0070e+00  9e-03  2e-06  1e-15
11: -9.9867e-01 -1.0053e+00  7e-03  1e-06  1e-15
12: -9.9913e-01 -1.0025e+00  3e-03  6e-07  1e-15
13: -9.9934e-01 -1.0015e+00  2e-03  3e-07  1e-15
14: -9.9954e-01 -1.0005e+00  9e-04  9e-08  1e-15
15: -9.9962e-01 -1.0002e+00  6e-04  4e-08  1e-15
16: -9.9970e-01 -9.9991e-01  2e-04  1e-08  1e-15
17: -9.9974e-01 -9.9980e-01  7e-05  4e-10  2e-15
18: -9.9975e-01 -9.9977e-01  2e-05  1e-17  2e-15
19: -9.9976e-01 -9.99

In [5]:
results_cross_val

{(<function kernel.scattering_with_haar_wavelets>, 0.1): []}

In [6]:
DF = pd.DataFrame(data=pd.Series(prediction), columns=['Prediction'])
DF.index += 1
DF.to_csv('../data/'+'scattering_wavelet_0_1.csv', index=True, index_label='Id', sep=',')

## Toy data for tests

In [ ]:
n_samples = 150
mean_1 = [0, -3]
cov = [[3, 0], [0, 3]]
X_1 = np.random.multivariate_normal(mean_1, cov, n_samples).T
mean_2 = [3, 3]
X_2 = np.random.multivariate_normal(mean_2, cov, n_samples).T
mean_3 = [-3, 3]
X_3 = np.random.multivariate_normal(mean_3, cov, n_samples).T
X = np.concatenate((X_1, X_2, X_3), axis = 1)

y = np.concatenate((np.zeros((1,n_samples)), np.ones((1, n_samples)), 2*np.ones((1,n_samples))), axis=1)
y = y[0,:]
Xtr_t = X.T
Ytr = y

mask_test = range(0,450, 5)
mask_train = [i for i in range(450) if i not in mask_test]

Xtr_train = Xtr_t[mask_train, :].T
Xtr_test = Xtr_t[mask_test, :].T
Ytr_train = Ytr[mask_train]
Ytr_test = Ytr[mask_test]